In [1]:
import os
os.getcwd()

'/home/ec2-user/SageMaker/AWSexample'

In [2]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
bucket_name = "examplecaodata"
myregion=boto3.session.Session().region_name
print(myregion)

us-east-2


In [4]:
#!aws s3 mb s3://examplecaodata --region us-east-2

In [5]:
# create s3 bucket data folder
s3 = boto3.resource('s3')
try:
    if myregion=="us-east-2":
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': myregion})
    print("S3 bucket created!")
except Exception as e:
    print("error:",e)

error: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [6]:
# output path
prefix="xgboost-as-a-built-in-algo"
output_path=f"s3://{bucket_name}/{prefix}/output"
print(output_path)

s3://examplecaodata/xgboost-as-a-built-in-algo/output


In [8]:
# load data
import pandas as pd
import urllib
data_url = "https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv"
try:
    urllib.request.urlretrieve(data_url, "bank_clean.csv")
except Exception as e:
    print("data load error:",e)

model_data = pd.read_csv("./bank_clean.csv",index_col=0)
model_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [14]:
# data split
from sklearn.model_selection import train_test_split
data = model_data.drop(['y_no', 'y_yes'], axis="columns")
labels = model_data['y_yes']
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.30, random_state=1729)

len(data_train),len(data_test)

(28831, 12357)

In [18]:
pd.concat([labels_train,data_train], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([labels_test,data_test], axis=1).to_csv('test.csv', index=False, header=False)

In [20]:
# upload data into s3 buckets
# declare the data path

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [22]:
! aws s3 ls {bucket_name}/{prefix}/ --recursive

2024-06-18 21:38:53    1519426 xgboost-as-a-built-in-algo/test/test.csv
2024-06-18 21:38:53    3545018 xgboost-as-a-built-in-algo/train/train.csv
